<h4>Import all required libraries & packages</h4>

In [ ]:
import ee
import os
import shutil
import time

from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


<h4>Authenticate at Google</h4>

(See [here](https://developers.google.com/earth-engine/guides/service_account) for how to authenticate)

In [ ]:
service_account = "<Your Service account email>"
path_private_key = "<path to your private key (json)"

# get the credentials
credentials = ee.ServiceAccountCredentials(service_account, path_private_key)

# authenticate and initialize Google Earth Engine
ee.Initialize(credentials)

<h4>Specify the parameters</h4>

In [ ]:
satellite_type = "S2"  # typ of satellite, currently only Sentinel-2
epsg_code = 3031  # epsg code for the crs
start_date  = "YYYY-MM-DD"  # must be in YYYY-MM-DD
end_date = "YYYY-MM-DD"  # must be in YYYY-MM-DD
month = 0 # specify the month of the composite, if 0 all months are used

filter_clouds = True
cloud_filter = 50  # Maximum image cloud cover percent allowed in image collection

output_folder = "<Specify path to output folder>"
output_name = "<Output name>"  # without the ending

min_x = 0
max_x = 0
min_y = 0
max_y = 0

<h4>Function to mask cloud pixels</h4>

In [ ]:
def mask_s2_clouds(image):
    qa = image.select('QA60')

    # left shift operator
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # create mask
    cloud_mask = qa.bitwiseAnd(cloud_bit_mask).eq(0)
    cirrus_mask = qa.bitwiseAnd(cirrus_bit_mask).eq(0)

    # combine masks
    mask = cloud_mask.And(cirrus_mask)

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).multiply(255)

In [ ]:
# specify collection for satellites
if satellite_type == "S2":
    collection_link = "COPERNICUS/S2"
else:
    collection_link = None  # noqa
    raise NotImplementedError

# create aoi
polygon_ee = [[[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y], [min_x, min_y]]]
aoi = ee.Geometry.Polygon(polygon_ee, f"EPSG:{epsg_code}", evenOdd=False)

# build a collection and filter based on AOI, date,
collection = (ee.ImageCollection(collection_link)
              .filterBounds(aoi)
              .filterDate(start_date, end_date))

# filter for month
if month > 0:
    collection = collection.filter(ee.Filter.calendarRange(month, month, 'month'))  # noqa

# filter for clouds if filter_clouds is true
if filter_clouds:
    collection = collection.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)).map(mask_s2_clouds)

# keep only certain bands
collection = collection.select(['B2', 'B3', 'B4'])

# Map the function over one year of data and take the median.
composite = collection.median()

# clip to region of interest
composite = composite.clip(aoi)

<h4>Export the file to the Gdrive of the Service account</h4>

(the only way to store files over 32MB)

In [ ]:
# options for export
export_config = {
    'scale': 10,
    'region': aoi,
    'crs': "EPSG:" + str(epsg_code),
    'maxPixels': 26549123161
}

task = ee.batch.Export.image.toDrive(composite, output_name, **export_config)
task.start()

print("Start exporting image to Google Drive")

# get start time of task
start = datetime.now()

# Monitor the task to wait until the data is downloaded.
while task.status()['state'] in ['READY', 'RUNNING']:  # still running

    # wait for x seconds
    time.sleep(30)

    # get time difference
    now = datetime.now()
    diff = now - start
    days, seconds = diff.days, diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60

    print(f"Task is still running ({hours}:{minutes}:{seconds})")

# if the exporting is finished
else:
    if task.status()['state'] == 'FAILED':
        print("The export failed. Please try again")
        print(task.status()["error_message"])
        exit()
    else:
        print("File exported successfully")

<h4>Get the files from the Gdrive</h4>

In [ ]:
# authenticate to Drive
g_auth = GoogleAuth()
g_auth.credentials = ServiceAccountCredentials

# save drive connection as variable
drive = GoogleDrive(g_auth)

# get all non deleted files in the drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

# iterate all these files
for file in file_list:

    # get filename
    file_name = file['title']

    # download file into working dir
    file.GetContentFile(file_name, mimetype="image/tiff")

    # delete file on Gdrive
    file.Delete()

    # get current location of file
    source_path = os.getcwd() + "/" + file_name

    # get new path
    destination_path = output_folder + "/" + file_name

    # copy file to final destination folder
    shutil.move(source_path, destination_path)

    print(f"Satellite image successfully exported to {destination_path}")